In [21]:
import pandas as pd
import sqlite3
import os

# ---  1. Definicion de Rutas ---
# Define la rutas del archivo CSV, el nombre de la tabla y Base de datos en SQLite

#Ruta
cleaned_csv = "../data/cleaned/cleaned_amazon_products.csv"
#Ruta de base de datos SQLite
db_name = '../data/amazon_productos.db'
#Nombre de la tabla
table_name = 'productos_amazon'

print(f"Ruta del CSV limpio: {cleaned_csv}")
print(f"Ruta del DB y su nombre: {db_name}")
print(f"Nombre de la tabla: {table_name}")

# --- 2. Carga de Datos Limpios a Pandas DataFrame ---
# Carga el archivo CSV limpio en un DataFrame de Pandas para poder manipularlo

df_cleaned = None
try:
    df_cleaned = pd.read_csv(cleaned_csv)
    print(f"\nDatos limpios cargados a Dataframe. Filas: {len(df_cleaned)}")
    print(f"\nPrimeras 5 filas del Data frame:")
    print(df_cleaned.head())
    print(f"\nInfo del Data frame:")
    df_cleaned.info()
except FileNotFoundError:
    print(f"El archivo no se encontro en: {cleaned_csv}.")
except Exception as e:
    print(f"\nOcurrió un error al cargar el CSV: {e}")

# --- 3. Conexión a la Base de Datos SQLite y Carga de Datos ---
# Establece una conexión con la base de datos SQLite. Luego, los datos del DataFrame se insertarán en la tabla SQL.

if df_cleaned is not None :
    conn = None
    try:
        print(f"Intentando conectar a la base de datos: {db_name}")
        conn = sqlite3.connect(db_name)
        print("Conexión a la base de datos establecida.")

        #Obtener el cursor
        print("Intentando obtener un cursor de la conexión...")
        cursor = conn.cursor()
        print("Cursor obtenido exitosamente.")

        # Cargar el DataFrame a la base de datos
        df_cleaned.to_sql(table_name, conn, if_exists="replace", index=False)

        print(f"\nDatos cargados exitosamente en la tabla '{table_name}' de la base de datos '{db_name}'.")

        # Verifica la carga listando las tablas y las columnas de la tabla creada
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        print("\nTablas en la base de datos:", cursor.fetchall())

        cursor.execute(f"PRAGMA table_info({table_name});")
        print(f"\nEsquema de la tabla '{table_name}':")
        for row in cursor.fetchall():
         print(row)
    except sqlite3.Error as e:
            print(f"\nError de SQLite durante la carga de datos: {e}")
    except Exception as e:
            print(f"\nOcurrió un error inesperado: {e}")
    finally:
            if conn: 
                conn.close()
                print("\nConexión a la base de datos cerrada.")
else:
    print("\nNo se pudo cargar el DataFrame; omitiendo la carga a la base de datos.")

#    --- 4. Exploración de Datos con Consultas SQL ---
# consultas para extraer información y realizar análisis exploratorios.

print(f"----  Realizando consultas Exploratorias en SQL  ----")

conn = None
try:
     conn = sqlite3.connect(db_name)
     print(f"Conexion a base de datos Exitosa")

     #Consulta 5 filas
     query_head = f"SELECT * FROM {table_name} LIMIT 5;"
     df_head = pd.read_sql_query(query_head, conn)
     print(f"Primeras 5 Filas ")
     print(df_head)

     #Total de Productos
     query_count = f"SELECT COUNT(product_id) FROM {table_name};"
     total_products = pd.read_sql_query(query_count, conn)
     print(f"\n2. Número total de productos: {total_products}")

     # Top 10 productos por 'rating_count' (productos más populares)
     query_top_rated = f"""
     SELECT 
        product_name,
        category,
        rating,
        rating_count
     FROM {table_name}
     WHERE 
        rating_count IS NOT NULL AND rating_count > 0
     ORDER BY 
        rating_count DESC
     LIMIT 10;
     """
     df_top_rated = pd.read_sql_query(query_top_rated, conn)
     print(f"\n3. Top 10 productos por clasificacion")
     print(df_top_rated)

     #Producto con el mayor descuento
     query_max_discount = f"""
     SELECT 
        product_name,
        category,
        discount_percentage,
        actual_price,
        discounted_price,
        absolute_discount
     FROM {table_name}
     WHERE 
        discount_percentage IS NOT NULL
     ORDER BY 
        discount_percentage DESC
     LIMIT 10;
     """
     df_max_discount = pd.read_sql_query(query_max_discount, conn)
     print(f"\n4. Top 10 productos con el mayor porcentaje de descuento:")
     print(df_max_discount)

     # Precios promedio, descuentos y calificaciones por categoría (Top 10 por número de productos)
     query_avg_category = f"""
     SELECT
        category,
        COUNT(*) AS Num_Products,
        ROUND(AVG(actual_price), 2) AS Avg_Price,
        ROUND(AVG(discounted_price), 2) AS Avg_discount,
        ROUND(AVG(discount_percentage), 2) AS Avg_Percentage_discount,
        ROUND(AVG(rating), 2) AS Avg_Rating,
        SUM(rating_count) AS total_rating_count_category
     FROM 
        {table_name}
     WHERE 
        category IS NOT NULL
     GROUP BY 
        category
     ORDER BY
        num_products DESC
     LIMIT 10;
     """
     df_avg_category = pd.read_sql_query(query_avg_category, conn)
     print("\n5. Precios promedio, descuentos y calificaciones por categoría (Top 10 por productos):")
     print(df_avg_category)

     #  Análisis de Descuento vs. Popularidad por rango de descuento
     query_discount_popularity = f"""
     SELECT
            CASE
                WHEN discount_percentage = 0 THEN '0% (Sin descuento)'
                WHEN discount_percentage BETWEEN 1 AND 10 THEN '01-10%'
                WHEN discount_percentage BETWEEN 11 AND 20 THEN '11-20%'
                WHEN discount_percentage BETWEEN 21 AND 30 THEN '21-30%'
                WHEN discount_percentage BETWEEN 31 AND 40 THEN '31-40%'
                WHEN discount_percentage BETWEEN 41 AND 50 THEN '41-50%'
                WHEN discount_percentage BETWEEN 51 AND 60 THEN '51-60%'
                WHEN discount_percentage BETWEEN 61 AND 70 THEN '61-70%'
                WHEN discount_percentage BETWEEN 71 AND 80 THEN '71-80%'
                WHEN discount_percentage BETWEEN 81 AND 90 THEN '81-90%'
                WHEN discount_percentage BETWEEN 91 AND 100 THEN '91-100%'
                ELSE 'Otros/Desconocido'
            END AS discount_range,
            COUNT(*) AS num_products,
            SUM(rating_count) AS total_rating_count_in_range,
            AVG(rating_count) AS avg_rating_count_per_product,
            AVG(rating) AS avg_rating
        FROM
            {table_name}
        WHERE
            discount_percentage IS NOT NULL 
        GROUP BY
            discount_range
        ORDER BY
            CASE
                WHEN discount_range = '0% (Sin descuento)' THEN 0
                WHEN discount_range = '01-10%' THEN 1
                WHEN discount_range = '11-20%' THEN 2
                WHEN discount_range = '21-30%' THEN 3
                WHEN discount_range = '31-40%' THEN 4
                WHEN discount_range = '41-50%' THEN 5
                WHEN discount_range = '51-60%' THEN 6
                WHEN discount_range = '61-70%' THEN 7
                WHEN discount_range = '71-80%' THEN 8
                WHEN discount_range = '81-90%' THEN 9
                WHEN discount_range = '91-100%' THEN 10
                ELSE 99
            END ASC;
        """
     df_discount_popularity = pd.read_sql_query(query_discount_popularity, conn)
     print("\n6. Análisis de Descuento vs. Popularidad (por rango de descuento):")
     print(df_discount_popularity)

      # Impacto de la Calificación Promedio en la Popularidad
     query_rating_impact = f"""
     SELECT
        CASE
            WHEN rating >= 4.5 THEN '4.5 - 5.0 (Excelente)'
            WHEN rating >= 4.0 THEN '4.0 - 4.4 (Muy Bueno)'
            WHEN rating >= 3.5 THEN '3.5 - 3.9 (Bueno)'
            WHEN rating >= 3.0 THEN '3.0 - 3.4 (Aceptable)'
            ELSE 'Menos de 3.0 (Bajo)'
        END AS rating_band,
        COUNT(*) AS num_products,
        SUM(rating_count) AS total_rating_count_in_band,
        ROUND(AVG(rating_count), 2) AS avg_rating_count_per_product,
        ROUND(AVG(discount_percentage), 2) AS avg_discount_percentage,
        ROUND(AVG(actual_price), 2) AS avg_actual_price
     FROM
        {table_name}
     WHERE
        rating IS NOT NULL 
     GROUP BY
        rating_band
     ORDER BY
        CASE
            WHEN rating_band = '4.5 - 5.0 (Excelente)' THEN 5
            WHEN rating_band = '4.0 - 4.4 (Muy Bueno)' THEN 4
            WHEN rating_band = '3.5 - 3.9 (Bueno)' THEN 3
            WHEN rating_band = '3.0 - 3.4 (Aceptable)' THEN 2
            ELSE 1
        END DESC;
     """
     df_rating_impact = pd.read_sql_query(query_rating_impact, conn)
     print("\n7. Impacto de la Calificación Promedio en la Popularidad (por banda de calificación):")
     print(df_rating_impact)

     # Top 10 Productos Populares SIN Descuento
     query_popular_no_discount = f"""
     SELECT
        product_name,
        category,
        rating,
        rating_count,
        actual_price
     FROM
        {table_name}
     WHERE
        (discount_percentage IS NULL OR discount_percentage = 0) AND rating_count IS NOT NULL AND rating_count > 0
     ORDER BY
        rating_count DESC,
        rating DESC
     LIMIT 10;
     """
     df_popular_no_discount = pd.read_sql_query(query_popular_no_discount, conn)
     print("\n11. Top 10 Productos Populares SIN Descuento:")
     print(df_popular_no_discount)

     # Consulta 10: Top 10 Productos Populares CON Descuento
     query_popular_discounted = f"""
     SELECT
        product_name,
        category,
        rating,
        rating_count,
        discount_percentage,
        actual_price,
        discounted_price,
        absolute_discount
     FROM
        {table_name}
     WHERE
        discount_percentage IS NOT NULL AND discount_percentage > 0 AND rating_count IS NOT NULL AND rating_count > 0
     ORDER BY
        rating_count DESC,
        discount_percentage DESC
     LIMIT 10;
     """
     df_popular_discounted = pd.read_sql_query(query_popular_discounted, conn)
     print("\n10. Top 10 Productos Populares CON Descuento:")
     print(df_popular_discounted)

  
except Exception as e :
     print(f"\nOcurrió un error durante la ejecución de las consultas SQL: {e}")
finally:
    # --- CERRAR LA CONEXIÓN ---
    if conn:
        conn.close()
        print("\nConexión a la base de datos cerrada.")

Ruta del CSV limpio: ../data/cleaned/cleaned_amazon_products.csv
Ruta del DB y su nombre: ../data/amazon_productos.db
Nombre de la tabla: productos_amazon

Datos limpios cargados a Dataframe. Filas: 1465

Primeras 5 filas del Data frame:
   product_id                                       product_name  \
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1  B098NS6PVG  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...   
2  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
3  B08HDJ86NZ  boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...   
4  B08CF3B7N1  Portronics Konnect L 1.2M Fast Charging 3A 8 P...   

                                            category  discounted_price  \
0  Computers&Accessories|Accessories&Peripherals|...             399.0   
1  Computers&Accessories|Accessories&Peripherals|...             199.0   
2  Computers&Accessories|Accessories&Peripherals|...             199.0   
3  Computers&Accessories|Accessories&Peripherals|...     